In [2]:
from loguru import logger
from mh_nlp.domain.services.text_cleaner import TextCleaner
from mh_nlp.infrastructure.data.kaggle_repository import KaggleDatasetRepository
from mh_nlp.infrastructure.nlp.tokenizers import SpacyTokenizerAdapter
from mh_nlp.application.use_cases.build_clean_dataset import BuildCleanDatasetUseCase


# 0. Définir le mapping des labels
label_mapping = {
    "Normal": 0,
    "Depression": 1,
    "Suicidal": 2,
    "Anxiety": 3,
    "Bipolar": 4,
    "Stress": 5,
    "Personality disorder": 6
}

filtered_label_mapping = {
    "Anxiety": 0, 
    "Normal": 1, 
    "Depression": 2,
}

# 1. Setup des composants (Wiring)
repo = KaggleDatasetRepository(
    csv_path="../data/raw/mental_health.csv",
    label_mapping=filtered_label_mapping,
)
cleaner = TextCleaner(engine=SpacyTokenizerAdapter(model="en_core_web_sm"))

# 2. Initialisation du Use Case
use_case = BuildCleanDatasetUseCase(repository=repo, cleaner=cleaner)

# 3. Exécution
dataset_final = use_case.execute()

# Maintenant, dataset_final.texts et dataset_final.labels sont prêts pour Sklearn ou PyTorch !

2026-01-15 12:49:11.694 | INFO     | mh_nlp.infrastructure.nlp.tokenizers:__init__:33 - SpacyTokenizerAdapter : Modèle 'en_core_web_sm' chargé avec succès.
2026-01-15 12:49:11.694 | DEBUG    | mh_nlp.domain.services.text_cleaner:__init__:37 - TextCleaner initialisé avec SpacyTokenizerAdapter (Batch mode: False)
2026-01-15 12:49:11.694 | DEBUG    | mh_nlp.domain.services.text_cleaner:__init__:39 - TextCleaner initialisé avec SpacyTokenizerAdapter
2026-01-15 12:49:11.696 | INFO     | mh_nlp.application.use_cases.build_clean_dataset:execute:32 - Application : Démarrage du Use Case BuildCleanDataset.
2026-01-15 12:49:11.696 | INFO     | mh_nlp.infrastructure.data.kaggle_repository:load:33 - Démarrage du chargement des données depuis : ../data/raw/mental_health.csv
2026-01-15 12:49:12.250 | SUCCESS  | mh_nlp.infrastructure.data.kaggle_repository:load:57 - Infrastructure : 2000 paires (Document, Label) créées.
2026-01-15 12:49:18.548 | SUCCESS  | mh_nlp.application.use_cases.build_clean_data

In [2]:
from mh_nlp.domain.entities.document import Document
from mh_nlp.infrastructure.nlp.fast_tokenizers import FastSpacyTokenizerAdapter
from mh_nlp.domain.services.fast_text_cleaner import FastTextCleaner

# 1. Initialisation
engine = FastSpacyTokenizerAdapter(model="en_core_web_sm", batch_size=10)
cleaner = FastTextCleaner(engine=engine)

# 2. Données de test (incluant un cas problématique : le vide)
data = [
    # 1. Cas Standard
    Document(text="I feel very depressed and lonely today."),
    
    # 2. Cas Vide (déjà testé, mais crucial pour l'alignement)
    Document(text=""),
    
    # 3. Espaces uniquement (doit être géré comme vide)
    Document(text="     "),
    
    # 4. Bruit Social Media massif
    Document(text="RT @user123: This is so sad!!! #mentalhealth #depression #help @charity_org"),
    
    # 5. URLs multiples et complexes
    Document(text="Found help here: https://support.org/help?id=123 and http://test.com/path"),
    
    # 6. Caractères spéciaux et ponctuation excessive
    Document(text="HELP ME !!!... ??? (I am not okay) [urgent] *sigh*"),
    
    # 7. Chiffres et dates (doivent être supprimés par ta regex [^a-z\s])
    Document(text="I have been suffering since 2010. 10/10 would not recommend."),
    
    # 8. Mélange Majuscules/Minuscules (Case Sensitivity)
    Document(text="vErY ANXIOUS AND sTrEsSeD out right now."),
    
    # 9. Texte avec sauts de ligne et tabulations
    Document(text="First line.\nSecond line with\ttabs."),
    
    # 10. Texte très court (un seul mot)
    Document(text="Sad."),
    
    # 11. Texte très long (pour tester la performance du batch)
    Document(text="I am " + "so " * 100 + "tired."),
    
    # 12. Caractères non-ASCII / Emojis (doivent être filtrés)
    Document(text="I am happy today! 😊🌟 Love life! ❤️"),
    
    # 13. Contenu purement numérique
    Document(text="1234567890"),
    
    # 14. Contenu purement ponctuation
    Document(text="!!! ??? @@@ ###"),
    
    # 15. Stopwords uniquement (doit retourner une chaîne vide après spaCy)
    Document(text="I am the and of a"),
    
    # 16. Mots avec apostrophes (Contractions)
    Document(text="I can't sleep, I'm exhausted and shouldn't stay up."),
    
    # 17. Balises HTML (si ton scraping est imparfait)
    Document(text="<p>This is a <b>test</b> of HTML removal.</p>"),
    
    # 18. Texte avec répétition de lettres (Slang)
    Document(text="I am soooooooo hhaapppyyyyyyy"),
    
    # 19. Format technique (Logs)
    Document(text="2023-01-01 12:00:00 [ERROR] User is feeling low"),
    
    # 20. Phrases interrogatives et exclamatives collées
    Document(text="IsThisReal?NoItIsnt!HelpMe.")
]

# 3. Test du traitement par lot
print("--- Début du Batch Cleaning ---")
results = cleaner.clean_batch(data)

for i, res in enumerate(results):
    print(f"Doc {i} nettoyé : '{res}'")

# Vérification de l'alignement
assert len(results) == len(data), "ERREUR : La taille du dataset a changé !"

2026-01-15 13:16:31.511 | INFO     | mh_nlp.infrastructure.nlp.fast_tokenizers:__init__:33 - SpacyTokenizerAdapter : Modèle 'en_core_web_sm' prêt (Batch: 10).
2026-01-15 13:16:31.511 | DEBUG    | mh_nlp.domain.services.fast_text_cleaner:__init__:24 - FastTextCleaner (Batch Only) prêt avec FastSpacyTokenizerAdapter
2026-01-15 13:16:31.513 | INFO     | mh_nlp.infrastructure.nlp.fast_tokenizers:process_batch:50 - Démarrage du traitement par lots (20 lignes).


--- Début du Batch Cleaning ---


Batch Lemmatization (spaCy): 100%|██████████| 20/20 [00:00<00:00, 557.60it/s]
2026-01-15 13:16:31.552 | SUCCESS  | mh_nlp.infrastructure.nlp.fast_tokenizers:process_batch:63 - Traitement massif terminé avec succès.


Doc 0 nettoyé : 'feel depressed lonely today'
Doc 1 nettoyé : ''
Doc 2 nettoyé : ''
Doc 3 nettoyé : 'rt sad'
Doc 4 nettoyé : 'find help'
Doc 5 nettoyé : 'help okay urgent sigh'
Doc 6 nettoyé : 'suffer recommend'
Doc 7 nettoyé : 'anxious stress right'
Doc 8 nettoyé : 'line second line tab'
Doc 9 nettoyé : 'sad'
Doc 10 nettoyé : 'tired'
Doc 11 nettoyé : 'happy today love life'
Doc 12 nettoyé : ''
Doc 13 nettoyé : ''
Doc 14 nettoyé : ''
Doc 15 nettoyé : 't sleep m exhausted shouldn t stay'
Doc 16 nettoyé : 'p b test b html removal p'
Doc 17 nettoyé : 'soooooooo hhaapppyyyyyyy'
Doc 18 nettoyé : 'error user feel low'
Doc 19 nettoyé : 'isthisreal noitisnt helpme'


In [1]:
from loguru import logger

# Infrastructure
from mh_nlp.infrastructure.data.kaggle_repository import KaggleDatasetRepository
from mh_nlp.infrastructure.nlp.fast_tokenizers import FastSpacyTokenizerAdapter

# Domaine (Services)
# Note : On utilise FastTextCleaner pour bénéficier du traitement par lots
from mh_nlp.domain.services.fast_text_cleaner import FastTextCleaner

# Application (Use Case & DTO)
from mh_nlp.application.use_cases.build_clean_dataset import BuildCleanDatasetUseCase

# --- CONFIGURATION ---

# Mapping des étiquettes métier vers les index numériques
FILTERED_LABEL_MAPPING = {
    "Anxiety": 0, 
    "Normal": 1, 
    "Depression": 2,
}

# --- 1. SETUP DES COMPOSANTS (WIRING) ---

# Initialisation du dépôt de données (Source : CSV Kaggle)
repo = KaggleDatasetRepository(
    csv_path="../data/raw/mental_health.csv",
    label_mapping=FILTERED_LABEL_MAPPING,
)

# Initialisation du moteur NLP haute performance (Port -> Adapter)
# batch_size=1000 permet d'optimiser les performances de spaCy sur CPU
spacy_engine = FastSpacyTokenizerAdapter(
    model="en_core_web_sm", 
    batch_size=1000
)

# Initialisation du service de nettoyage (Orchestrateur Domaine)
cleaner = FastTextCleaner(engine=spacy_engine)

# --- 2. INITIALISATION DU USE CASE ---

# Le Use Case reçoit ses dépendances par injection
use_case = BuildCleanDatasetUseCase(
    repository=repo, 
    cleaner=cleaner
)

# --- 3. EXÉCUTION ---

logger.info("Démarrage du pipeline de préparation des données...")

# Lancement du processus complet : Chargement -> Nettoyage Batch -> DTO
dataset_final = use_case.execute()

# --- 4. VÉRIFICATION ---

if dataset_final.total_processed > 0:
    logger.success(f"Dataset prêt pour l'entraînement ! Taille : {dataset_final.total_processed}")
    print(f"Exemple du premier document : {dataset_final.documents[0][:100]}...")

2026-01-15 13:50:19.097 | INFO     | mh_nlp.infrastructure.nlp.fast_tokenizers:__init__:33 - SpacyTokenizerAdapter : Modèle 'en_core_web_sm' prêt (Batch: 1000).
2026-01-15 13:50:19.098 | DEBUG    | mh_nlp.domain.services.fast_text_cleaner:__init__:24 - FastTextCleaner (Batch Only) prêt avec FastSpacyTokenizerAdapter
2026-01-15 13:50:19.098 | INFO     | __main__:<module>:51 - Démarrage du pipeline de préparation des données...
2026-01-15 13:50:19.099 | INFO     | mh_nlp.application.use_cases.build_clean_dataset:execute:48 - Application : Lancement du pipeline de nettoyage de données.
2026-01-15 13:50:19.099 | INFO     | mh_nlp.infrastructure.data.kaggle_repository:load:33 - Démarrage du chargement des données depuis : ../data/raw/mental_health.csv
2026-01-15 13:50:19.556 | SUCCESS  | mh_nlp.infrastructure.data.kaggle_repository:load:57 - Infrastructure : 2000 paires (Document, Label) créées.
2026-01-15 13:50:19.564 | INFO     | mh_nlp.application.use_cases.build_clean_dataset:execute:68

Exemple du premier document : oh gosh...


In [ ]:

from mh_nlp.infrastructure.nlp.fast_tokenizers import FastSpacyTokenizerAdapter

spacy_engine = FastSpacyTokenizerAdapter(model="en_core_web_sm",batch_size=1000)
print(getattr(spacy_engine, "batch_size", None) is not None)

filtered_label_mapping = {
    "Anxiety": 0, 
    "Normal": 1, 
    "Depression": 2,
}

# 1. Setup des composants (Wiring)
repo = KaggleDatasetRepository(
    csv_path="../data/raw/mental_health.csv",
    label_mapping=filtered_label_mapping,
)
spacy_engine = FastSpacyTokenizerAdapter(model="en_core_web_sm",batch_size=1000)
cleaner = TextCleaner(engine=SpacyTokenizerAdapter(model="en_core_web_sm"))

# 2. Initialisation du Use Case
use_case = BuildCleanDatasetUseCase(repository=repo, cleaner=cleaner)

# 3. Exécution
dataset_final = use_case.execute()

2026-01-15 13:19:51.159 | INFO     | mh_nlp.infrastructure.nlp.fast_tokenizers:__init__:33 - SpacyTokenizerAdapter : Modèle 'en_core_web_sm' prêt (Batch: 1000).
2026-01-15 13:19:51.573 | INFO     | mh_nlp.infrastructure.nlp.tokenizers:__init__:33 - SpacyTokenizerAdapter : Modèle 'en_core_web_sm' chargé avec succès.
2026-01-15 13:19:51.573 | DEBUG    | mh_nlp.domain.services.text_cleaner:__init__:37 - TextCleaner initialisé avec SpacyTokenizerAdapter (Batch mode: False)
2026-01-15 13:19:51.574 | DEBUG    | mh_nlp.domain.services.text_cleaner:__init__:39 - TextCleaner initialisé avec SpacyTokenizerAdapter
2026-01-15 13:19:51.574 | INFO     | mh_nlp.application.use_cases.build_clean_dataset:execute:32 - Application : Démarrage du Use Case BuildCleanDataset.
2026-01-15 13:19:51.575 | INFO     | mh_nlp.infrastructure.data.kaggle_repository:load:33 - Démarrage du chargement des données depuis : ../data/raw/mental_health.csv
2026-01-15 13:19:52.031 | SUCCESS  | mh_nlp.infrastructure.data.kagg

In [11]:
# main.py
from mh_nlp.domain.services.text_cleaner import TextCleaner
from mh_nlp.domain.entities.document import Document
from mh_nlp.infrastructure.nlp.tokenizers import SpacyTokenizerAdapter, NltkTokenizerAdapter

raw_text = "The students are studying and checking their emails at https://univ.edu !"
document = Document(raw_text)

# --- CAS 1 : Utilisation de spaCy (Lemmatisation précise) ---
spacy_engine = SpacyTokenizerAdapter(model="en_core_web_sm")
cleaner_spacy = TextCleaner(engine=spacy_engine)
print(f"SpaCy Result: {cleaner_spacy.clean(document)}")
# Sortie attendue: "student study check email"


2026-01-15 11:18:49.004 | INFO     | mh_nlp.infrastructure.nlp.tokenizers:__init__:32 - SpacyTokenizerAdapter : Modèle 'en_core_web_sm' chargé avec succès.
2026-01-15 11:18:49.005 | DEBUG    | mh_nlp.domain.services.text_cleaner:__init__:33 - TextCleaner initialisé avec SpacyTokenizerAdapter


SpaCy Result: student study check email


In [ ]:
# --- CAS 2 : Utilisation de NLTK avec Stemming (Plus agressif) ---
nltk_engine = NltkTokenizerAdapter(method="stem")
cleaner_nltk = TextCleaner(engine=nltk_engine)
print(f"NLTK Stem Result: {cleaner_nltk.clean(raw_text)}")
# Sortie attendue: "student studi check email" (Note le 'studi' tronqué)

# --- CAS 3 : Utilisation de NLTK avec Lemmatisation ---
nltk_lem = NltkTokenizerAdapter(method="lemmatize")
cleaner_lem = TextCleaner(engine=nltk_lem)
print(f"NLTK Lemma Result: {cleaner_lem.clean(raw_text)}")

In [ ]:
from mh_nlp.infrastructure.data.kaggle_repository import KaggleDatasetRepository

# 1. Définir le mapping des labels (nom -> index numérique)
label_mapping = {
    "Normal": 0,
    "Depression": 1,
    "Suicidal": 2,
    "Anxiety": 3,
    "Bipolar": 4,
    "Stress": 5,
    "Personality disorder": 6
}

# 2. Charger les données
repo = KaggleDatasetRepository(
    csv_path="../data/raw/mental_health.csv",
    label_mapping=label_mapping
)
raw_data = repo.load()

# 3. Transformer en dataset exploitable
dataset = []
for document, label in raw_data:
    dataset.append({
        'text': document.text,
        'label': label.name,
        'label_id': label.index
    })

# Afficher quelques exemples
print(f"Nombre total d'exemples : {len(dataset)}")
print("\nPremiers exemples :")
for i, item in enumerate(dataset[:3]):
    print(f"\nExemple {i+1}:")
    print(f"  Texte: {item['text'][:100]}...")
    print(f"  Label: {item['label']} (ID: {item['label_id']})")

In [ ]:
# Alternative : format simplifié
dataset = [
    (document.text, label.index) 
    for document, label in raw_data
]

# Exemple d'utilisation
texts, labels = zip(*dataset)
print(f"Nombre de textes : {len(texts)}")
print(f"Premier texte : {texts[0]}")
print(f"Premier label : {labels[0]}")

In [ ]:
# Séparer textes et labels
texts = [document.text for document, label in raw_data]
labels = [label.index for document, label in raw_data]
label_names = [label.name for document, label in raw_data]

print(f"Nombre d'exemples : {len(texts)}")
print(f"\nExemple 1:")
print(f"  Texte: {texts[0]}")
print(f"  Label: {label_names[0]} ({labels[0]})")

In [12]:
import spacy
import re
from tqdm import tqdm

# 1. Chargement optimisé du modèle
# On désactive le 'parser' (syntaxe) et le 'ner' (entités nommées) pour gagner en vitesse
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def fast_clean_spacy(texts):
    cleaned_texts = []
    
    # Prétraitement léger par RegEx avant d'envoyer à spaCy
    # On le fait sous forme de générateur pour économiser la mémoire
    def pre_process(t_list):
        for text in t_list:
            text = text.lower()
            text = re.sub(r'http\S+|www.\S+', '', text)
            text = re.sub(r'@\w+|#\w+', '', text)
            text = re.sub(r'[^a-z\s]', '', text)
            yield text

    # 2. Utilisation de nlp.pipe avec tqdm pour la barre de progression
    # nlp.pipe est beaucoup plus rapide que .apply() car il traite par lots (batches)
    for doc in tqdm(nlp.pipe(pre_process(texts), batch_size=500), 
                    total=len(texts), 
                    desc="Nettoyage spaCy"):
        
        # Lemmatisation et retrait des stopwords / espaces
        words = [token.lemma_ for token in doc if not token.is_stop and not token.is_space]
        cleaned_texts.append(' '.join(words))
        
    return cleaned_texts

# --- Application au DataFrame ---
# --- Données de test ---
test_sentences = [
    "I am running in the park and I love it! #happy", 
    "Check out this website: https://openai.com for more info.",
    "The cats are sitting on the mats. @someone",
    "Better late than never, but never late is better.",
    "I've been studying NLP with spaCy for 3 hours!"
]

# --- Exécution du test ---
print("Lancement du test...")
cleaned_results = fast_clean_spacy(test_sentences)

# --- Affichage comparatif ---
for original, cleaned in zip(test_sentences, cleaned_results):
    print(f"\nORIGINAL : {original}")
    print(f"NETTOYÉ  : {cleaned}")

Lancement du test...


Nettoyage spaCy: 100%|██████████| 5/5 [00:00<00:00, 94.08it/s]


ORIGINAL : I am running in the park and I love it! #happy
NETTOYÉ  : run park love

ORIGINAL : Check out this website: https://openai.com for more info.
NETTOYÉ  : check website info

ORIGINAL : The cats are sitting on the mats. @someone
NETTOYÉ  : cat sit mat

ORIGINAL : Better late than never, but never late is better.
NETTOYÉ  : well late late well

ORIGINAL : I've been studying NLP with spaCy for 3 hours!
NETTOYÉ  : have study nlp spacy hour


In [13]:
import random

# --- Générateur de 100 lignes de test ---
templates = [
    "I am running in the park with my {} dogs! #nature",
    "Check this link {} for more details on {} @user",
    "The {} is sitting on the {}, it's so {}!",
    "Learning {} with spaCy is {} than NLTK.",
    "I've been {} for {} hours now, it's {}."
]

words = ["cat", "data", "better", "running", "machine learning", "fast", "website", "amazing"]
urls = ["http://test.com", "https://example.org", "www.nlp-cool.io"]

# Création de la liste de 100 phrases
test_sentences_100 = []
for i in range(100):
    tpl = random.choice(templates)
    # On remplit les trous {} avec des mots aléatoires pour varier
    if tpl.count("{}") == 1:
        sentence = tpl.format(random.choice(words))
    elif tpl.count("{}") == 2:
        sentence = tpl.format(random.choice(urls), random.choice(words))
    else:
        sentence = tpl.format(random.choice(words), random.choice(words), random.choice(words))
    test_sentences_100.append(sentence)

# --- Exécution avec la fonction fast_clean_spacy ---
print(f"Traitement de {len(test_sentences_100)} lignes...")
results = fast_clean_spacy(test_sentences_100)

# Affichage des 5 premiers résultats pour vérification
print("\n--- Aperçu des résultats (5 premiers) ---")
for i in range(5):
    print(f"Ligne {i+1} : {results[i]}")

Traitement de 100 lignes...


Nettoyage spaCy: 100%|██████████| 100/100 [00:00<00:00, 336.81it/s]


--- Aperçu des résultats (5 premiers) ---
Ligne 1 : website sit amazing cat
Ligne 2 : run park data dog
Ligne 3 : learn spacy machine learning nltk
Ligne 4 : have cat well hour fast
Ligne 5 : learn spacy machine learning nltk


In [14]:
import pandas as pd
import numpy as np

# Construction d'un dataset hétérogène
raw_texts = [
    "The AI revolution is transforming software engineering globally.", # Court
    "Natural Language Processing (NLP) allows machines to understand human speech." * 20, # Long/Répétitif
    "I'm running, he runs, she ran; we've been running for miles in the rain.", # Conjugaisons (Test Lemme)
    "Contact us at support@example.com or visit https://nlp.spacy.io for docs.", # Entités/URL
    "This is a very simple sentence." # Très court
]

# On multiplie pour atteindre 5000 lignes (suffisant pour voir la différence de perf)
dataset = raw_texts * 1000 
print(f"Dataset prêt : {len(dataset)} lignes.")

Dataset prêt : 5000 lignes.


In [ ]:
import time
import psutil
import os
import pandas as pd
import matplotlib.pyplot as plt
from typing import List

# Simulation d'un dataset de test (remplacez par vos données réelles)
test_data = [
    "The quick brown fox jumps over the lazy dog and looks for more adventure." * 10
] * 1000  # 1000 documents de taille moyenne

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)  # Retourne en Mo

def run_benchmark(adapter, data: List[str], name: str):
    print(f"--- Benchmarking: {name} ---")
    
    start_mem = get_memory_usage()
    start_time = time.time()
    
    # Exécution du traitement par lots
    results = adapter.process_batch(data)
    
    end_time = time.time()
    end_mem = get_memory_usage()
    
    duration = end_time - start_time
    mem_diff = end_mem - start_mem
    throughput = len(data) / duration
    
    return {
        "Modèle": name,
        "Temps total (s)": round(duration, 3),
        "Docs/seconde": round(throughput, 2),
        "Mémoire utilisée (Mo)": round(mem_diff, 2)
    }

# 1. Initialisation des adaptateurs
spacy_adapter = SpacyTokenizerAdapter(model="en_core_web_sm", batch_size=1000)
#nltk_adapter = NltkTokenizerAdapter(method="lemmatize")

# 2. Lancement des tests
results = []
results.append(run_benchmark(spacy_adapter, test_data, "spaCy (Optimized)"))
results.append(run_benchmark(nltk_adapter, test_data, "NLTK (WordNet)"))

# 3. Affichage des résultats
df_results = pd.DataFrame(results)
print("\nTableau Comparatif :")
display(df_results)

# 4. Visualisation
df_results.set_index("Modèle")[["Temps total (s)", "Docs/seconde"]].plot(
    kind="bar", subplots=True, figsize=(10, 6), title="Performance Comparison"
)
plt.tight_layout()
plt.show()

In [15]:
import pandas as pd
import numpy as np

# Construction d'un dataset hétérogène
raw_texts = [
    "The AI revolution is transforming software engineering globally.", # Court
    "Natural Language Processing (NLP) allows machines to understand human speech." * 20, # Long/Répétitif
    "I'm running, he runs, she ran; we've been running for miles in the rain.", # Conjugaisons (Test Lemme)
    "Contact us at support@example.com or visit https://nlp.spacy.io for docs.", # Entités/URL
    "This is a very simple sentence." # Très court
]

# On multiplie pour atteindre 5000 lignes (suffisant pour voir la différence de perf)
dataset = raw_texts * 1000 
print(f"Dataset prêt : {len(dataset)} lignes.")

import time
import spacy
from tqdm import tqdm

def benchmark_spacy_configs(texts):
    results = []
    
    # --- CONFIG 1 : NO OPTIMIZED (Unitaire + Tous composants) ---
    nlp_full = spacy.load("en_core_web_sm")
    start = time.time()
    res_1 = [ [t.lemma_ for t in nlp_full(txt) if not t.is_stop] for txt in tqdm(texts, desc="Mode: Unitaire Full") ]
    results.append({"Configuration": "Unitaire (Full Pipeline)", "Temps (s)": time.time() - start})

    # --- CONFIG 2 : SEMI OPTIMIZED (Batch + Tous composants) ---
    start = time.time()
    res_2 = []
    for doc in tqdm(nlp_full.pipe(texts, batch_size=1000), total=len(texts), desc="Mode: Batch Full"):
        res_2.append([t.lemma_ for t in doc if not t.is_stop])
    results.append({"Configuration": "Batch (Full Pipeline)", "Temps (s)": time.time() - start})

    # --- CONFIG 3 : FULL OPTIMIZED (Batch + Disabled Components) ---
    # C'est ce que fait votre classe SpacyTokenizerAdapter
    nlp_fast = spacy.load("en_core_web_sm", disable=["ner", "parser"])
    start = time.time()
    res_3 = []
    for doc in tqdm(nlp_fast.pipe(texts, batch_size=1000), total=len(texts), desc="Mode: Batch Optimized"):
        res_3.append([t.lemma_ for t in doc if not t.is_stop])
    results.append({"Configuration": "Batch (Optimized: No NER/Parser)", "Temps (s)": time.time() - start})

    return pd.DataFrame(results)

# Exécution
df_bench = benchmark_spacy_configs(dataset)
display(df_bench)

Dataset prêt : 5000 lignes.


Mode: Batch Optimized: 100%|██████████| 5000/5000 [00:33<00:00, 149.14it/s]


,Configuration,Temps (s)
0,Unitaire (Full Pipeline),92.085369
1,Batch (Full Pipeline),71.288010
2,Batch (Optimized: No NER/Parser),33.527106
